In [31]:
import numpy as np

def is_positive_definite(A, num_tests=10):
    n = A.shape[0]
    for _ in range(num_tests):
        x = np.random.randn(n)
        val = x.T @ A @ x
        if val <= 0:
            return False
    return True

def is_diagonally_dominant(A):
    return np.all(np.abs(np.diag(A)) > np.sum(np.abs(A), axis=1) - np.abs(np.diag(A)))

def make_diagonally_dominant(A, b):
    A, b = A.copy(), b.copy()
    n = len(A)
    for i in range(n):
        for j in range(i, n):
            if abs(A[j, i]) > np.sum(np.abs(A[j, :])) - abs(A[j, i]):
                if i != j:
                    A[[i, j]] = A[[j, i]]
                    b[[i, j]] = b[[j, i]]
                break
    return A, b

def jacobi(A, b, eps=1e-6, max_iter=10000, x0=None):
    A = np.array(A, dtype=float)
    b = np.array(b, dtype=float)
    n = A.shape[0]
    
    if x0 is None:
        x = np.zeros(n)
    else:
        x = np.array(x0, dtype=float)
        
    D = np.diag(A)
    
    if np.any(np.isclose(D, 0.0)):
        raise ZeroDivisionError("Діагональні елементи містять нулі.")
        
    R = A - np.diagflat(D)
    history = []

    D1 = np.diag(np.diag(A))
    L = np.tril(A, k=-1)
    U = np.triu(A, k=1)
    #print(D1)
    #print(L)
    #print(U)
    for k in range(1, max_iter+1):
        x_new = (b - R.dot(x)) / D
        diff = np.max(np.abs(x_new - x))
        if k <= 10:
            history.append(x_new.copy())
        x = x_new
        if diff < eps:
            res = np.max(np.abs(A.dot(x) - b))
            return x, {"converged": True, "iterations": k, "residual_norm": res, "history": history}
    res = np.max(np.abs(A.dot(x) - b))
    return x, {"converged": False, "iterations": max_iter, "residual_norm": res, "history": history}
    
def write_output(x, info, A, filename="output.txt"):
    cond_number = np.linalg.cond(A)
    with open(filename, "w") as f:
        f.write("Розв'язок системи:\n")
        f.write(" ".join(f"x{i+1} = {xi:.10f}" for i, xi in enumerate(x)) + "\n")
        f.write(f"Збіг методу: {info['converged']}\n")
        f.write(f"Число обумовленості матриці: {cond_number:.4e}\n")
        f.write(f"Кількість ітерацій: {info['iterations']}\n")
        f.write(f"Норма нев'язки ||Ax-b||_inf: {info['residual_norm']:.10e}\n")
        f.write("\nПерші ітерації (до 10):\n")
        for k, xi in enumerate(info["history"], 1):
            f.write(f"Ітерація {k}: " + " ".join(f"x{i+1} = {val:.10f}" for i, val in enumerate(xi)) + "\n")

def main():
    data = np.loadtxt("input.txt", dtype=float)
    A = data[:, :-1]
    b = data[:, -1]
    
    if not is_diagonally_dominant(A):
        A, b = make_diagonally_dominant(A, b)
    
    x, info = jacobi(A, b, eps=1e-10, max_iter=100000, x0=b)    
    write_output(x, info, A,"output.txt")
    
if __name__ == "__main__":
    main()

In [54]:
def jacobi_apriori_via_C(A, b, eps=1e-6, norm_type=1):
    A = np.array(A, dtype=float)
    b = np.array(b, dtype=float)
    
    D = np.diag(A)
    if np.any(np.isclose(D, 0.0)):
        raise ZeroDivisionError("Діагональні елементи містять нулі.")

    if not is_diagonally_dominant(A):
        A, b = make_diagonally_dominant(A, b)
    
    L_plus_R = A - np.diagflat(D)
    
    C = L_plus_R / D[:, None]
    
    C_norm = np.linalg.norm(C, ord=norm_type)

    if C_norm > 0.5:
        raise ValueError("||C|| > 0.5")
    
    x0 = np.zeros_like(b)
    x1 = b / D  
    initial_norm = np.linalg.norm(x1 - x0, ord=norm_type)
    
    k_est = np.ceil(np.log(eps * (1 - C_norm) / initial_norm) / np.log(C_norm))
    
    return int(k_est), C_norm


In [50]:
data = np.loadtxt("input.txt", dtype=float)
A = data[:, :-1]
b = data[:, -1]
jacobi_apriori_via_C(A,b)

ValueError: ||C|| > 0.5